In [ ]:
import numpy as np
import pandas as pd
import linearsolve as ls
import matplotlib.pyplot as plt
plt.style.use('classic')
plt.rcParams['figure.facecolor'] = 'white'

# Class 14: Prescott's Real Business Cycle Model II

In this notebook, we continue to examine the centralized version of the model from pages 11-17 in Edward Prescott's article "Theory Ahead of Business Cycle Measurement in the Fall 1986 of the Federal Reserve Bank of Minneapolis' *Quarterly Review* (link to article: [https://www.minneapolisfed.org/research/qr/qr1042.pdf](https://www.minneapolisfed.org/research/qr/qr1042.pdf)). In this notebook, we:

1. Look at the effect of changing $\rho$ (the autoregressive coefficient on log TFP) on the simulated impulse responses of model variables to a TFP shock
2. Compute a stochastic simulation of the model and compute summary statistics.

## Example: Effect of Changing $\rho$ on Impulse Responses

Recall that the equilibrium conditions for Prescott's RBC model are:

\begin{align}
\frac{1}{C_t} & = \beta E_t \left[\frac{\alpha A_{t+1}K_{t+1}^{\alpha-1}L_{t+1}^{1-\alpha} +1-\delta }{C_{t+1}}\right]\\
\frac{\varphi}{1-L_t} & = \frac{(1-\alpha)A_tK_t^{\alpha}L_t^{-\alpha}}{C_t} \\
Y_t & = A_t K_t^{\alpha}L_t^{1-\alpha}\\
K_{t+1} & = I_t + (1-\delta) K_t\\
Y_t & = C_t + I_t\\
\log A_{t+1} & = \rho \log A_t + \epsilon_{t+1}
\end{align}

where $\epsilon_{t+1} \sim \mathcal{N}(0,\sigma^2)$. 

The objective is use `linearsolve` to simulate impulse responses to a TFP shock for $\rho = 0.5,0.75,0.9,0.99$. Other parameter values are given in the table below:

| $$\sigma$$ | $$\beta$$ | $$\varphi$$ | $$\alpha$$ | $$\delta $$ |
|------------|-----------|-------------|------------|-------------|
| 0.006      | 0.99      | 1.7317      | 0.35       |  0.025      |

## Model Preparation

As usual, we recast the model in the form required for `linearsolve`. Write the model with all variables moved to the right-hand side of the equations and dropping the expecations operator $E_t$ and the exogenous shock $\epsilon_{t+1}$:

\begin{align}
0 & = \beta\left[\frac{\alpha A_{t+1}K_{t+1}^{\alpha-1}L_{t+1}^{1-\alpha} +1-\delta }{C_{t+1}}\right] - \frac{1}{C_t}\\
0 & = \frac{(1-\alpha)A_tK_t^{\alpha}L_t^{-\alpha}}{C_t} - \frac{\varphi}{1-L_t}\\
0 & = A_t K_t^{\alpha}L_t^{1-\alpha} - Y_t\\
0 & = I_t + (1-\delta) K_t - K_{t+1}\\
0 & = C_t + I_t - Y_t\\
0 & = \rho \log A_t - \log A_{t+1}
\end{align}

Remember, capital and TFP are called *state variables* because their period $t+1$ values are determined by their period $t$ values. TFP is an *exogenous state variable* because it's $t+1$ value is subject to an exogenous shock $\epsilon_{t+1}$. Capital is an *endogenous state variable* because the $t+1$ value is completely determined in date $t$. Output, consumption, and investment are called a *costate* or *control* variables. Note that the model has 5 equations in 5 endogenous variables.

## Initialization, Approximation, and Solution

The next several cells initialize the model in `linearsolve` and then approximate and solve it.

In [ ]:
# Create a variable called `parameters` that stores the model parameter values in a Pandas Series. CELL PROVIDED
# Note that a value for rho is absent
parameters = pd.Series(dtype=float)

parameters['beta'] = 0.99
parameters['phi'] = 1.7317
parameters['alpha'] = 0.35
parameters['delta'] = 0.025

# Print the model's parameters
print(parameters)

In [ ]:
# Create a variable called `sigma` that stores the value of sigma. CELL PROVIDED
sigma = 0.006

In [ ]:
# Create variable called `exo_states` that stores the names of each exogenous state variable. CELL PROVIDED
exo_states = ['a']

# Create variable called `endo_states` that stores the names of each endogenous state variable.
endo_states = ['k']

# Create variable called `costates` that stores the names of the non-predetermined variables.
costates = ['y','c','i','l']

In [ ]:
# Define a function that evaluates the equilibrium conditions of the model solved for zero. CELL PROVIDED
def equations(variables_forward,variables_current,parameters):
    
    # Parameters.
    p = parameters
    
    # Current variables.
    cur = variables_current
    
    # Forward variables.
    fwd = variables_forward

    # Euler equation
    mpk = p.alpha*fwd.a*fwd.k**(p.alpha-1)*fwd.l**(1-p.alpha)
    euler_equation = p.beta*(mpk+1-p.delta)/fwd.c - 1/cur.c
    
    # Labor-leisure choice
    mpl = (1-p.alpha)*cur.a*cur.k**p.alpha*cur.l**(-p.alpha)
    labor_leisure = mpl/cur.c - p.phi/(1-cur.l)
    
    # Production function
    production_function = cur.a*cur.k**p.alpha*cur.l**(1-p.alpha) - cur.y
    
    # Capital evolution
    capital_evolution = cur.i + (1 - p.delta)*cur.k - fwd.k
    
    # Market clearing
    market_clearing = cur.c+cur.i - cur.y
    
    # Exogenous tfp
    tfp_process = p.rho*np.log(cur.a) - np.log(fwd.a)
    
    # Stack equilibrium conditions into a numpy array
    return np.array([
        euler_equation,
        labor_leisure,
        production_function,
        capital_evolution,
        market_clearing,
        tfp_process
        ])

Next, initialize the model using `ls.model()` which takes the following required arguments:

* `equations`
* `exo_states`
* `endo_states`
* `costates`
* `parameters`

In [ ]:
# Initialize the model into a variable named `rbc_model`. CELL PROVIDED.
rbc_model = ls.model(equations = equations,
                     exo_states=exo_states,
                     endo_states=endo_states,
                     costates=costates,
                     parameters=parameters)

## SImulation and Plotting

The objective is to create a $2\times 2$ grid of plots containing the impulse responses of TFP, output, labor, and consumption to a one percent shock to TFP for each of the values for $\rho$: 0.5,0.75,0.9,0.99. Here are the steps that we'll take:

1. Initilize figure and axes for plotting.
2. Iterate over each desired value for $\rho$.
    1. Set `rbc_model.parameters['rho']` equal to current value of $\rho$.
    2. Use `rbc_model.compute_ss()` to compute the steady state with initial guess of $A=1$, $K=4$, $Y=1$, $C=1$, $L=0.5$, and $I=1$.
    3. Use `rbc_model.approximate_and_solve()` to approximate and solve the model with the current value of $\rho$.
    4. Use `rbc_model.impulse()` to compute the 51 period impulse response to a 0.01 unit shock to TFP in period 5.
    5. Add the computed impulse responses to the axes.

In [ ]:
# Create a 12x8 figure PROVIDED
fig = plt.figure(figsize=(12,8))

# Create four axis variables: `ax1`, `ax2`, `ax3`, `ax4`
# First axis. PROVIDED
ax1 = fig.add_subplot(2,2,1)
# Second axis. PROVIDED
ax2 = fig.add_subplot(2,2,2)
# Third axis. PROVIDED
ax3 = fig.add_subplot(2,2,3)
# Fourth axis. PROVIDED
ax4 = fig.add_subplot(2,2,4)

# Create an axis equal to the size of the figure. PROVIDED
ax0 = fig.add_subplot(1,1,1)
# Turn off the axis so that the underlying axes are visible. PROVIDED
ax0.axis('off')

# Create variable called 'rho_values' that stores the desired values of rho


# Set the guess for the steady state


# Iterate over the elements of rho_values


    # Update the value of rho in rbc_model.parameters


    # Compute the steady state with initial guess defined above


    # Approximate the model and solve


    # Compute the impulse responses to a 0.01 unit shock to TFP


    # Add plots of TFP, output, labor, and consumption to ax1, ax2, ax3, and ax4


    # Plot the point 0,0 on ax0 with the same line properties used for the other plotted lines and provide a label


# Set axis 1 title. PROVIDED
ax1.set_title('TFP')

# Set axis 2 title. PROVIDED
ax2.set_title('Output')

# Set axis 3 title. PROVIDED
ax3.set_title('Labor')

# Set axis 4 title. PROVIDED
ax4.set_title('Consumption')

# Add grid to the axis 1. PROVIDED
ax1.grid()

# Add grid to the axis 2. PROVIDED
ax2.grid()

# Add grid to the axis 3. PROVIDED
ax3.grid()

# Add grid to the axis 4. PROVIDED
ax4.grid()

# Set ax1 y-axis limits to [0,2]. PROVIDED
ax1.set_ylim([0,2])
# Set ax2 y-axis limits to [0,2]. PROVIDED
ax2.set_ylim([0,2])
# Set ax3 y-axis limits to [-0.5,1.25]. PROVIDED
ax3.set_ylim([-0.5,1.5])
# Set ax4 y-axis limits to [-0.5,1.5]. PROVIDED
ax4.set_ylim([-0.5,1.5])

# Add legend below the figure. PROVIDED
legend = ax0.legend(loc='upper center',bbox_to_anchor=(0.5,-0.075), ncol=4,fontsize=15)

## Example: Stochastic Simulation

Compute a 401 period stochastic simulation of the model. Set $\rho=0.75$ to match US business cycle data. And set the seed for the simulation to 126.

In [ ]:
# Set the value of rho in `rbc_model.parameters` to 0.75


# Compute the steady state with initial guess defined above


# Approximate the model and solve


In [ ]:
# Compute the stochastic simulation using the .stoch_sim() method of rbc_model


On a single axis, plot the simulated values for output, consumption, investment, and labor.

In [ ]:
# Create 12x4 figure. PROVIDED
fig = plt.figure(figsize=(12,4))

# Create axis. PROVIDED
ax = fig.add_subplot(1,1,1)

# Plot output, consumption, investment, labor


# Y-axis label. PROVIDED
ax.set_ylabel('% dev from ss')

# Add legend outside plot. PROVIDED
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Add a grid. PROVIDED
ax.grid()

In [ ]:
# Display the standard deviations (times 100) of output, consumption, investment, and labor in the simulated data


In [ ]:
# Display the correlations of output, consumption, investment, and labor in the simulated data
